In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
os.chdir("/content/drive/MyDrive/code/growth")
os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive/code/growth'

In [2]:
import numpy as np
from sklearn.metrics import mean_squared_error
from scipy import optimize
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
plt.rcParams['font.sans-serif'] = ['Arial'] 
import scipy.optimize
import pandas as pd
import seaborn as sns
import pickle
import math

In [3]:
#import data 
data = pd.read_csv("data/bayesian_fitting.csv")
TIME = data['TIME']
N = data['Nt']
N_new = N *(10 ** data['Dilution'])
data['N_new'] = N_new
N_new = data['N_new']
N0_objective = data['N0_objective']
Initial = data['Initial']

In [4]:
#parameter estimation
#fitting 
import pystan
from pystan import StanModel

stan_code = """
  data {
    int K;
    real TIME[K];
    int<lower=0> N_new[K];
    real N0_objective[K];
    int Initial[K]; 
  }

  transformed data{
  	int<lower=0> N0[K];
	  for (k in 1:K) {
		  N0[k] = poisson_rng(N0_objective[k]); 	
	  }
  }

  parameters {
    real<lower=0> mu;
    real<lower=0> lag;
    real<lower=0> n0;
  }

  transformed parameters {
      real<lower=0> F[K];	
      for (k in 1:K) {
          F[k] = exp((-mu) * (TIME[k]-lag));
		  if (TIME[k] <= lag) {
			  F[k] = 0;
		  }
		  else {
			  F[k] = exp((-mu) * (TIME[k]-lag));
		  }
	  }
  }

  model {
	  for (k in 1:K) {
		  if (Initial[k] != 0) {
			  N_new[Initial[k]] ~ poisson(n0);
		  }
		  if (Initial[k] == 0 ) {
			  if ( N_new[k] >= N0[k] ) {
				  (N_new[k]-N0[k]) ~ neg_binomial(n0, F[k]) ; 
			  }
			  else {
				  N_new[k] ~ poisson(n0);
			  }	
		  }
	  }
  }
"""

model = pystan.StanModel(model_code=stan_code)
with open('data/model-growth.pkl','wb') as f:
    pickle.dump(model,f)


INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_f65ce50889876e262cc62d55bcdbc84f NOW.


In [5]:
stan_data = data.to_dict('list')
stan_data.update({'K':len(data)})

#MCMC sampling
fit_nuts = model.sampling(data = stan_data, chains=4, iter=10000, warmup=5000)
with open('data/result-growth.pkl','wb') as g:
    pickle.dump(fit_nuts,g)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  import sys


In [6]:
fit_nuts

Inference for Stan model: anon_model_f65ce50889876e262cc62d55bcdbc84f.
4 chains, each with iter=10000; warmup=5000; thin=1; 
post-warmup draws per chain=5000, total post-warmup draws=20000.

        mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
mu      0.71  7.6e-6 7.7e-4   0.71   0.71   0.71   0.71   0.72  10393    1.0
lag     2.72  1.3e-4   0.01   2.69   2.71   2.72   2.72   2.74   7906    1.0
n0    6354.9    0.35  31.96 6292.6 6333.3 6354.8 6376.7 6416.9   8391    1.0
F[1]     0.0     nan    0.0    0.0    0.0    0.0    0.0    0.0    nan    nan
F[2]     0.0     nan    0.0    0.0    0.0    0.0    0.0    0.0    nan    nan
F[3]     0.0     nan    0.0    0.0    0.0    0.0    0.0    0.0    nan    nan
F[4]     0.0     nan    0.0    0.0    0.0    0.0    0.0    0.0    nan    nan
F[5]     0.0     nan    0.0    0.0    0.0    0.0    0.0    0.0    nan    nan
F[6]     0.0     nan    0.0    0.0    0.0    0.0    0.0    0.0    nan    nan
F[7]     0.4  3.4e-5 3.0e-3   0.39    0